- target size in load_img [224, 224]
- different pooling
- include_top = True
- PCA size versuchen
- standardscaler nochmals versuchen
- different order of training set

In [34]:
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import glob
from tqdm import tqdm

import tensorflow

In [35]:
from keras.preprocessing import image

def get_vector(image_path, model):
        img = image.load_img(image_path, target_size=(224, 224))
        input_arr = image.img_to_array(img)
        input_arr = np.array([input_arr])
        return model.predict(input_arr)

In [36]:
sorted_img = sorted(glob.glob('data' + '/*.jpg'))

In [37]:
from keras.applications.efficientnet import EfficientNetB1

e1_model = EfficientNetB1(weights='imagenet', include_top=False, pooling='avg')
e1_feature_list = [get_vector(item, e1_model) for item in tqdm(sorted_img)]

100%|██████████| 10000/10000 [14:33<00:00, 11.45it/s]


In [101]:
from sklearn.decomposition import PCA

size = 20
pca = PCA(n_components=size, random_state=2)

e1_fla = [item.flatten() for item in e1_feature_list]
li_1 = pca.fit_transform(np.array(e1_fla))

10000


In [121]:
file = open('train_triplets.txt')

df = pd.DataFrame()

temp_list = []

counter = 0

for line in tqdm(file):
    
    words = line.split()
    
    index_x = int(words[0])
    index_y1 = int(words[1])
    index_y2 = int(words[2])
    
    x = li_1[index_x]
    y1 = li_1[index_y1]
    y2 = li_1[index_y2]
    
    
    if counter % 2 == 0:
        temp_list.append(np.concatenate((x, y1, y2), axis=None).reshape(3*size))
    else:
        temp_list.append(np.concatenate((x, y2, y1), axis=None).reshape(3*size))
    
    counter += 1

X = pd.DataFrame(temp_list)

59515it [00:00, 162023.23it/s]


In [120]:
y = pd.DataFrame(np.resize([1,0], 59515))

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)

In [115]:
clf = MLPClassifier(early_stopping=True, random_state=69, tol= 1e-5)
clf.fit(X_train, np.ravel(y_train))
score = clf.score(X_test, np.ravel(y_test))
print(score)

0.7106854838709677


8:
    0.704133064516129
    0.68331594359
18:
    0.7259744623655914
    0.698327218875
19:
    0.7326948924731183
    0.697182861566
20:
    0.7296706989247311
    0.69937060348
21:
    0.7281586021505376
    0.68876846959
22:
    0.7328629032258065
    0.685705630911
32:
    0.7286626344086021
    
64:
    0.735383064516129
    
512:
    0.8146841397849462
    0.664501363132

In [116]:
testFile = open('test_triplets.txt')

test_X = pd.DataFrame()

temp_list = []

for line in tqdm(testFile):
    
    words = line.split()
    
    index_x = int(words[0])
    index_y1 = int(words[1])
    index_y2 = int(words[2])
    
    x = li_1[index_x]
    y1 = li_1[index_y1]
    y2 = li_1[index_y2]
    
    temp_list.append(np.concatenate((x, y1, y2), axis=None).reshape(3*size))

test_X = pd.DataFrame(temp_list)

59544it [00:00, 178687.38it/s]


In [141]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(early_stopping=True, random_state=69, verbose=True, tol= 1e-5, alpha=0.1, beta).fit(X, np.ravel(y))
test_y = clf.predict(test_X)

Iteration 1, loss = 0.60394118
Validation score: 0.701109
Iteration 2, loss = 0.57315946
Validation score: 0.704133
Iteration 3, loss = 0.56861236
Validation score: 0.712534
Iteration 4, loss = 0.56618962
Validation score: 0.715894
Iteration 5, loss = 0.56331421
Validation score: 0.713206
Iteration 6, loss = 0.56165920
Validation score: 0.710349
Iteration 7, loss = 0.56070021
Validation score: 0.717574
Iteration 8, loss = 0.55970611
Validation score: 0.714046
Iteration 9, loss = 0.55917075
Validation score: 0.717070
Iteration 10, loss = 0.55757478
Validation score: 0.710013
Iteration 11, loss = 0.55708000
Validation score: 0.716734
Iteration 12, loss = 0.55613754
Validation score: 0.720766
Iteration 13, loss = 0.55593201
Validation score: 0.719590
Iteration 14, loss = 0.55475685
Validation score: 0.719254
Iteration 15, loss = 0.55585385
Validation score: 0.716398
Iteration 16, loss = 0.55505180
Validation score: 0.726815
Iteration 17, loss = 0.55433891
Validation score: 0.709509
Iterat

In [142]:
print(sum(test_y))

27253


In [143]:
np.savetxt('answer.txt', test_y, fmt='%d')